<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
     <img style="float: right; padding-right: 10px" width="100" src="https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/images/clemson_paw.png"> </div>
     </div>

**Clemson University**<br>
**Fall 2024**<br>
**Instructor(s):** Aaron Masino <br>

## Homework 6: Introduction to Neural Networks, PyTorch & PyTorch Lightning
This homework is intended to assess your knowledge of feed forward artificial neural networks and implementation using PyTorch and PyTorch Lightning. As presented in class, these Python libraries provide support for neural network, including deep-learning, model development and analysis. For complete information, you may reference:
-  Python documentation [here](https://www.python.org/)
-  PyTorch documentation [here](https://pytorch.org/)
-  PyTorch Lightning documentation[here](https://lightning.ai/docs/pytorch/stable/)


# Setup Instructions
Execute the first two code cells to import the required Python packages create the necessary data directories.

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torchvision.transforms.functional as F 

import lightning as L
from lightning.pytorch import seed_everything
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import torchmetrics as TM

from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
########### DO NOT MODIFY THIS CODE #############
def create_data_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

dir_dataroot = os.path.join("..", "data")
create_data_directory(dir_dataroot)

dir_lightning = os.path.join("..", "lightning")
create_data_directory(dir_lightning)

SEED = 123456

# Excercise 1 (2 points)
In the code cell below, assign to the variable `t` a two dimensional (2-D) PyTorch Tensor with 5 rows (first dimension) and 3 columns (second dimension) of random numbers selected from a uniform distribution on $\left[100,110\right)$.

In [ ]:
########### START YOUR CODE HERE #############


# Excercise 2 (1 point)
In the code cell below, use Tensor slicing to select and print the first two rows of the PyTorch Tensor `t`. Print ONLY the first two rows.

In [ ]:
########### DO NOT MODIFY THIS CODE #############
t = torch.rand(3,10)

########### START YOUR CODE HERE #############


# Exercise 3 (1 point)
In the code cell below, create a PyTorch Tensor, `t_dp` containg the dot product of each row (first dimension) of the tensors `t1` and `t2`. Hint: It is acceptable to use a `for` loop in your solution, however, a more computationally efficient solution will use the [torch.sum](https://pytorch.org/docs/stable/generated/torch.sum.html) method. Your final results should be a 1-D Tensor with 25 elements.

In [ ]:
t1 = torch.rand(25,5)
t2 = torch.rand(25,5)

############ START YOUR CODE HERE #############


# Exercise 4 (1 point)
In the code cell below, complete the function `can_be_matmul` which should return `True` if the input tensors `t_left` and `t_right` can be multiplied (in the matrix multiplication sens) and `False` otherwise. Your solution should first check if each Tensor is exactly two-dimensional and if the dimensions are appropriate for matrix multiplication.

In [ ]:
def can_be_matmul(t_left, t_right):
    ############ START YOUR CODE HERE #############
    


In the remaining exercises, you will be working with the __Fashion MNIST__ dataset. Fashion-MNIST is a dataset consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes related to the type of clothing article shown in the image. For more information on this dataset, see [https://github.com/zalandoresearch/fashion-mnist](Fashion MNIST github).

Execute the code in the following cell to load the train and test data from the the Fashion MNIST dataset. You should not modify this code.

In [ ]:
########### DO NOT MODIFY THIS CODE #############
fashion_train_all = FashionMNIST(root=dir_dataroot, download=True, train=True, transform=ToTensor())
fashion_test = FashionMNIST(root=dir_dataroot, download=True, train=False, transform=ToTensor())

def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

show(make_grid(next(iter(DataLoader(fashion_train_all, batch_size=16, shuffle=True)))[0], nrow=4))

seed_everything(SEED)
batch_size = 16

val_split = 0.2
val_size = int(len(fashion_train_all) * val_split)
train_size = len(fashion_train_all) - val_size
fashion_train, fashion_val = torch.utils.data.random_split(fashion_train_all, [train_size, val_size])

print("Training samples:",fashion_train_all.data[fashion_train.indices].shape)
print("Unique classes in training:",fashion_train_all.targets[fashion_train.indices].unique())

print("\nValidation samples:", fashion_train_all.data[fashion_val.indices].shape)
print("Unique classes in validation:", fashion_train_all.targets[fashion_val.indices].unique())

print("\nTest samples:",fashion_test.data.shape)
print(fashion_test.targets.unique())



# Exercise 5 (1 point)
In the code cell below, use the PyTorch `DataLoader` class to construct three data loaders, one each for the `fashion_train`, `fashion_val` and the `fashion_test` datasets. 

In [ ]:
############ START YOUR CODE HERE #############
train_loader = None
val_loader = None
test_loader = None

# Excercise 6 (1 point)
In the code cell below, complete the `make_encoder` function. This function will instantiate an instance of the `FashionEncoder` class based on the inputs:
- `sample_image` : a single image that is a two dimensional (2-D) tensor
- `hidden_size` : the desired number of hidden units
- `num_classes` : the number of classes in the dataset

Your solution for the `make_encoder` function should use the `sample_image` size to determine the value of the `input_dim` needed to create the `FashionEncoder` instance and assign it to teh `encoder` variable which is returned by the function.

In [ ]:
########### DO NOT MODIFY THIS CODE #############
class FashionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        x = self.relu(x)
        return x
    

def make_encoder(sample_image, num_hidden, num_classes):
    encoder = None
    ############ START YOUR CODE HERE #############
    
    
    ############ END YOUR CODE HERE #############
    return encoder

# Exercise 7 (2 points)
In the code cell below, the variable `model` is an instance of the `UntrainedModel` class. This is a fully conected feed forward neural network that has not been trained on any data (it's weights are randomly initialized). Use the model to predict the class label for the set of sample images in the `images` variable. Print the predicted class labels. Print the true labels which are contained in the variable, `labels`. You should expect the model predictions are wrong for most of the samples because it has not been trained (it might get lucky on some).

In [ ]:
########### DO NOT MODIFY THIS CODE #############
class FashionOutput(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super().__init__()
        self.linear = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        return self.linear(x)
    
class UntrainedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = FashionEncoder(28*28, 16, 10) 
        self.output = FashionOutput(16, 10)

    def forward(self, x):
        x = self.encoder(x)
        x = self.output(x)
        return x
    
model = UntrainedModel()
images, labels = next(iter(train_loader))

############ START YOUR CODE HERE #############


# Exercise 8 (1 point)
In the code cell below, the class `FashionClassifier` has already been implemented. In the next code cell, an instance of the classifer has been created. For this exercise, create an instance of the PyTorch Lightning `Trainer` class. Use the trainer to train the model on the training data.

In [ ]:
########### DO NOT MODIFY THIS CODE #############
class FashionClassifier(L.LightningModule):
    def __init__(self, encoder, output, num_classes):
        super().__init__()
        self.encoder = encoder
        self.output = output
        
       # validation metrics - we will use these to compute the metrics at the end of the validation epoch
        self.val_metrics_tracker = TM.wrappers.MetricTracker(TM.MetricCollection([TM.classification.MulticlassAccuracy(num_classes=num_classes)]), maximize=True)
        self.validation_step_outputs = []
        self.validation_step_targets = []

        # test metrics - we will use these to compute the metrics at the end of the test epoch
        self.test_roc = TM.ROC(task="multiclass", num_classes=num_classes) # roc and cm have methods we want to call so store them in a variable
        self.test_cm = TM.ConfusionMatrix(task='multiclass', num_classes=num_classes)
        self.test_metrics_tracker = TM.wrappers.MetricTracker(TM.MetricCollection([TM.classification.MulticlassAccuracy(num_classes=num_classes), 
                                                            self.test_roc, self.test_cm]), maximize=True) 
        
        # test outputs and targets - we will store the outputs and targets for the test step
        self.test_step_outputs = []
        self.test_step_targets = []

    # the forward method applies the encoder and output to the input
    def forward(self, x):
        x = self.encoder(x)
        x = self.output(x)
        return x

    # the training step. pass the batch through the model and compute the loss
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        # where is our softmax? We don't need it here because we are using cross_entropy which includes the softmax for efficiency
        loss = nn.functional.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss
    
    # the validation step. pass the batch through the model and compute the loss. Store the outputs and targets for the epoch end step and log the loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = nn.functional.cross_entropy(logits, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True)
        
        # store the outputs and targets for the epoch end step
        self.validation_step_outputs.append(logits)
        self.validation_step_targets.append(y)
        return loss
    
    # the test step. pass the batch through the model and compute the loss. Store the outputs and targets for the epoch end step and log the loss
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = nn.functional.cross_entropy(logits, y)
        self.log('test_loss', loss, on_step=True, on_epoch=True)
        self.test_step_outputs.append(logits)
        self.test_step_targets.append(y)
        return loss
    
    # at the end of the epoch compute the metrics
    def on_validation_epoch_end(self):
        # stack all the outputs and targets into a single tensor
        all_preds = torch.vstack(self.validation_step_outputs)
        all_targets = torch.hstack(self.validation_step_targets)
        
        # compute the metrics
        loss = nn.functional.cross_entropy(all_preds, all_targets)
        self.val_metrics_tracker.increment()
        self.val_metrics_tracker.update(all_preds, all_targets)
        self.log('val_loss_epoch_end', loss)
        
        # clear the validation step outputs
        self.validation_step_outputs.clear()
        self.validation_step_targets.clear()
    
    def on_test_epoch_end(self):
        all_preds = torch.vstack(self.test_step_outputs)
        all_targets = torch.hstack(self.test_step_targets)
        
        self.test_metrics_tracker.increment()
        self.test_metrics_tracker.update(all_preds, all_targets)
        # clear the test step outputs
        self.test_step_outputs.clear()
        self.test_step_targets.clear()

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

Exercise 8: Add your code to the notebook cell below to train the FashionClassifier model on the training data.

In [ ]:

########### DO NOT MODIFY THIS CODE #############
seed_everything(SEED)
model = FashionClassifier(FashionEncoder(28*28, 16, 10), 
                          FashionOutput(16, 10), 
                          num_classes=10)

############ START YOUR CODE HERE #############
trainer = None

# Optional
If you are interested, exectue the following code cells to evaluate model test set performance.

In [ ]:
trainer.test(model=model, dataloaders=test_loader)
rslt = model.test_metrics_tracker.compute()

In [ ]:
cmp = sns.heatmap(rslt['MulticlassConfusionMatrix'], annot=True, fmt='d', cmap='Blues')
cmp.set_xlabel('Predicted Label')
cmp.set_xticklabels(fashion_train_all.classes, rotation=90)
cmp.set_yticklabels(fashion_train_all.classes, rotation=0)
cmp.set_ylabel('Actual Label');

In [ ]:
fpr, tpr, thresholds = rslt['MulticlassROC']
for i in range(10):
    plt.plot(fpr[i], tpr[i], label=fashion_train_all.classes[i])
plt.xlabel('False Positive Rate')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid()